In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import xlrd

def remove(data):
    removal = []
    for i in removal:
        data = data[data["Course Code"] != i]
    return data
    
# Naming convention to be updated
def standardize(data):
    #Naming Convention
    convention = {'AET 304':'AET', 
                  'CH 104M': 'Chemistry Lab',
                  'CH 301': "Chemistry",
                  'CS 302':'Computer Science',
                  'EDIT 2318':'TTU Computer Science',
                  'GEO 302E':'Geoscience',
                  'GEOG 1401':'TTU Geoscience',
                  'HIS 315K':'HIST 1301',
                  'HIS 315L':'HIST 1302',
                  'M 301':'College Algebra',
                  'M 305G':'Precalculus',
                  'MATH 1550':'TTU Precalculus',
                  'MATH 2300':'TTU Statistics',
                  'PHY 302K':'PHYS 1301',
                  'PHY 302L':'PHYS 1302',
                  'PHYS 1403':'TTU Physics I',
                  'PHYS 1404':'TTU Physics II',
                  'RHE 306':'ENGL 1301',
                  'RHE 309K':'ENGL 1302',
                  'SDS 302':'Statistics',
                  "NSC 309": "NSC 309"}
    convention = pd.Series(convention)
    
    
    Course_Name_Complete = {"Thriving in our Digital World": "Thriving in our Digital World",
        "Intro to Rhetoric": "Introduction to Rhetoric: Reading, Writing, and Research",
        "Earth, Wind, and Fire": "Earth, Wind, and Fire: An Introduction to Geoscience",
        "United States, 1492-1865": "The United States: 1492-1865",
        "United States Since 1865": "The United States Since 1865",
        "Discovery Precalculus": "Discovery Precalculus: A Creative and Connected Approach",
        "Mechanics, Heat, and Sound": "Mechanics, Heat, and Sound",
        "EM, Optics, & Nuclear Physics": "Electromagnetism, Optics, and Nuclear Physics",
        "Rhe of Amer Iden": "Reading and Writing the Rhetoric of American Identity",
        "OnRamps Statistics": "OnRamps Statistics",
        "Arts & Entertainment":"Pixels, Samples, Lumens, Illusion: Foundations of Arts & Entertainment Technologies",
        "College Algebra":"College Algebra",
        "Intro to Chem Practices (Lab)":"Introduction to Chemistry Practices I",
        "Principles of Chem (Lecture)":"Principles of Chemistry I"}
    Course_Name_Complete = pd.Series(Course_Name_Complete)
    
    TCCN = {"CS 302": "Computer Science",
        "RHE 306": "English Language Arts (ENGL 1301)",
        "RHE 309K": "English Language Arts (ENGL 1302)",
        "GEO 302E": "Geoscience",
        "HIS 315K": "US History (HIST 1301)",
        "HIS 315L": "US History (HIST 1302)",
        "M 305G": "Precalculus (MATH 2312)",
        "MATH 1550": "MATH 1550",
        "MATH 2300": "MATH 1550",
        "PHY 302K": "Physics (PHYS 1301)",
        "PHY 302L": "Physics (PHYS 1302)",
        "PHY 1403": "PHY 1403",
        "SDS 302": "Statistics",
        "AET 304": "Arts & Entertainment Technologies (AET)",
        "CH 301": "Chemistry (CH 1311)*",
        "CH 104M": "Chemistry Lab (CH 1111)*",
        "M 301": "College Algebra (MATH 1314)"
       }
    TCCN = pd.Series(TCCN)
    
    data["TCCN"] = data["Course Code"]
    for i in TCCN.index:
        data.loc[data["Course Code"] == i, "TCCN"] = TCCN[i]
    data["Course Title Complete"] = data["Course Title"]
    for i in Course_Name_Complete.index:
        data.loc[data["Course Title"] == i, "Course Title Complete"] = Course_Name_Complete[i]
    for i in convention.index:
        data.loc[data["Course Code"] == i, "Course Title"] = convention[i]
    return data

#Load Data
def get_master(master, district = ''):
    data = pd.read_csv(master)
    data = remove(data)
    data = data[data["Eligibility Status Final Simple"] != 'HS Only']
    data = standardize(data)
    if district == '':
        return data
    return data[data["District Name"] == district]

def read_data(master, district):
    data = get_master(master, district)
    UT = data[data["Partner"] == "UT Austin OnRamps"]
    TTU = data[data["Partner"] == "TTU OnRamps"]
    return UT, TTU

def read_data_without_differentiate(master, district):
    data = get_master(master, district)
    return data

# Get Row totals
def get_row_total(data):
    data['Total'] = 0
    for index, row in data.iterrows():
        data.loc[index, 'Total'] = sum(data.loc[index])
    return data

# Get Column totals
def get_column_total(data):
    data.loc['Total'] = 0
    for i in data.columns:
        data.loc['Total', i] = data[i].sum()
    return data

#Figure 1.1
def get_course(data, label = []):
    if label == '':
        course = data["Course Code"]
    else:
        course = data[label]
    return course.drop_duplicates()

def Figure_1_1(data):
    data = get_course(data, ["TCCN", "Course Title Complete", "Course Code", "Credit Hours"])
    data.columns = ["Subject & TCCN", "OnRamps Course Title", "UT Austin Course Code", "College Credit Hours"]
    data = data.sort_values("Subject & TCCN")
    data["College Credit Hours"] = data["College Credit Hours"].astype("int32")
    return data

def Figure_1_1_QC(data, district_name):
    template = [['', '', '', '', '', '', '', ''],
        ['', '1.0 ENROLLMENT', '', '', '', '', '', ''],
        ['', '', '', '', '', '', '', ''],
        ['', 'Figure 1.1 shows the OnRamps courses offered by the district in the 2018-2019 academic year.', '', '', '', '', '', ''],
        ['', '', '', '', '', '', '', ''],
        ['', '', f'Figure 1.1: OnRamps courses offered in {district_name} in the 2018-2019 academic year', None, None, None, None, ''],
        ['', '', 'Subject & TCCN', 'OnRamps Course Title', 'UT Austin Course Code', 'College Credit Hours', None, '']
        ]
    for i in range(len(data)):
        lrow = ['', '']
        for j in range(len(data.columns)):
            lrow.append(str(data.iloc[i,j]))
        lrow.append(None)
        lrow.append('')
        template.append(lrow)
    template.append(['', '', '*The Chemistry course and the lab are two separate courses at UT Austin. Students may be eligible for college credit in one, both, or neither course. The same is true for earning credit.', '', '', '', '', ''])
    return template

def verify(inputFile, QCData, index):
    loc = (inputFile)
    wb = xlrd.open_workbook(loc) 
    sheet = wb.sheet_by_index(index)
    QCCompleted = 1
    for i in range(sheet.nrows):
        if sheet.row_values(i) != QCData[i]:
            print(inputFile + " sheet " + str(index) + " " + str(i) + " Error!")
            print("EOY Output:")
            print(sheet.row_values(i))
            print("QC Output:")
            print(QCData[i])
            QCCompleted = 0
    return True

In [3]:
def report_generator_UT_TTU(master, district):
    
    OnRamps = read_data_without_differentiate(master, district)
    UT, TTU = read_data(master, district)
    # [Table_Generation_Function, DataBase, QC_Function]
    funclist = [(Figure_1_1, UT, Figure_1_1_QC),
               ]
    
    for i in funclist:
        func, data, QC = i
        return verify(district + ".xlsx", QC(func(data), district), 1)

In [5]:
master = "18-19 FAKE Master File (Final).csv"
district = pd.read_csv("DistrictName2.csv", header = None)

for i in range(len(district)):
    distrct_name = district.loc[i, 0]
    try:
        if report_generator_UT_TTU(master, distrct_name) == True:
            if i < len(district) - 1:
                district.loc[i + 1 :].to_csv("district not finished.csv", index=False)
    except FileNotFoundError as error:
        print(distrct_name)
        print(error)

In [6]:
import xlrd
loc = ("Austin ISD.xlsx") 
wb = xlrd.open_workbook(loc) 
sheet = wb.sheet_by_index(1) 
for i in range(sheet.nrows):
    print(sheet.row_values(i))

['', '', '', '', '', '', '', '']
['', '1.0 ENROLLMENT', '', '', '', '', '', '']
['', '', '', '', '', '', '', '']
['', 'Figure 1.1 shows the OnRamps courses offered by the district in the 2018-2019 academic year.', '', '', '', '', '', '']
['', '', '', '', '', '', '', '']
['', '', 'Figure 1.1: OnRamps courses offered in Austin ISD in the 2018-2019 academic year', None, None, None, None, '']
['', '', 'Subject & TCCN', 'OnRamps Course Title', 'UT Austin Course Code', 'College Credit Hours', None, '']
['', '', 'Arts & Entertainment Technologies (AET)', 'Pixels, Samples, Lumens, Illusion: Foundations of Arts & Entertainment Technologies', 'AET 304', '3', None, '']
['', '', 'Chemistry (CH 1311)*', 'Principles of Chemistry I', 'CH 301', '3', None, '']
['', '', 'Chemistry Lab (CH 1111)*', 'Introduction to Chemistry Practices I', 'CH 104M', '1', None, '']
['', '', 'College Algebra (MATH 1314)', 'College Algebra', 'M 301', '3', None, '']
['', '', 'Computer Science', 'Thriving in our Digital World